In [1]:
import json
import pandas as pd

In [56]:
import os
import glob

# Base directory containing all folders
base_directory = "./generated_answers/gpt-4-turbo-2024-04-09/test_must_why"

# Use glob to find all JSONL files recursively
jsonl_files = glob.glob(os.path.join(base_directory, "**", "*.jsonl"), recursive=True)

# Example: Print all found JSONL file paths
#print(jsonl_files)

In [80]:
# List of JSONL files
files = [['dependent_real_after_basic_binary_explain.jsonl', 'dependent_real_before_basic_binary_explain.jsonl'], ['nondependent_real_after_basic_binary_explain.jsonl', 'nondependent_real_before_basic_binary_explain.jsonl']]
files_to_use = files[1]

In [81]:
#Filter the list of found files based on your selection
selected_files = [file for file in jsonl_files if os.path.basename(file) in files_to_use]

# Example: Print selected file paths
print(selected_files)

['./generated_answers/gpt-4-turbo-2024-04-09/test_must_why/nondependent_real_before_basic_binary_explain.jsonl', './generated_answers/gpt-4-turbo-2024-04-09/test_must_why/nondependent_real_after_basic_binary_explain.jsonl']


In [82]:
data = []

for file_path in selected_files:
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))

# Convert the collected data into a DataFrame
df = pd.DataFrame(data)

In [83]:
df

,plan_idx,title,question_idx,steps,question_type,step_pair_idx_asked_about,binary_question,why_question,model_name,prompt_key,model_input,model_answer
0,710,spiced-guava-cake,710,"[Preheat oven to 180 C / Gas 4., Grease and fl...",nondependent_real_before,"[3, 4]",Must Step 4 happen before Step 5?,Why must Step 4 happen before Step 5?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","{\n ""binary_answer"": ""no"",\n ""why_answer"": ""..."
1,711,chocolate-chocolate-cake,711,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_before,"[1, 5]",Must Step 2 happen before Step 6?,Why must Step 2 happen before Step 6?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_..."
2,712,chicken-prawn-and-rice-soup,712,"[Mix together the prawns, chicken, egg and cor...",nondependent_real_before,"[3, 10]",Must Step 4 happen before Step 11?,Why must Step 4 happen before Step 11?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","{\n ""binary_answer"": ""no"",\n ""why_answer"": ""..."
3,713,green-tofu-curry,713,"[Place water in a medium saucepan, and stir in...",nondependent_real_before,"[0, 4]",Must Step 1 happen before Step 5?,Why must Step 1 happen before Step 5?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""no"",\n ""why_a..."
4,714,very-vanilla-ry-cheesecake,714,[Preheat oven to 150 C / Gas 2 and lightly but...,nondependent_real_before,"[4, 7]",Must Step 5 happen before Step 8?,Why must Step 5 happen before Step 8?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1415,1415,chocolate-chocolate-cake,1415,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_after,"[2, 18]",Must Step 19 happen after Step 3?,Why must Step 19 happen after Step 3?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""no"",\n ""why_a..."
1416,1416,chocolate-chocolate-cake,1416,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_after,"[5, 18]",Must Step 19 happen after Step 6?,Why must Step 19 happen after Step 6?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""no"",\n ""why_a..."
1417,1417,chicken and vegetable pie,1417,"[Preheat oven to 220 C / Gas mark 7., In a sau...",nondependent_real_after,"[3, 5]",Must Step 6 happen after Step 4?,Why must Step 6 happen after Step 4?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","{\n ""binary_answer"": ""yes"",\n ""why_answer"": ..."
1418,1418,chocolate-chocolate-cake,1418,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_after,"[10, 14]",Must Step 15 happen after Step 11?,Why must Step 15 happen after Step 11?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""no"",\n ""why_a..."


In [61]:
import ast

In [84]:
def clean_model_answer(text):
    try:
        # Remove newline characters and extra backslashes
        text = text.replace('\n', '').replace('\\', '').replace("json", "").strip("```")
        # Ensure the JSON string is properly closed
        if not text.endswith('}'):
            text += '}'
        # Parse the string as JSON
        data = json.loads(text)
        # Extract the 'binary_answer' and 'why_answer'
        binary_answer = data.get('binary_answer', '')
        why_answer = data.get('why_answer', '')
        return pd.Series([binary_answer, why_answer])
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        print(f"Problematic text: {text}")
        return pd.Series(['', ''])
    except Exception as e:
        print(f"Unexpected error: {e}")
        print(f"Problematic text: {text}")
        return pd.Series(['', ''])

In [85]:
# Apply the cleaning function to the 'model_answer' column
df[['binary_answer', 'why_answer']] = df['model_answer'].apply(clean_model_answer)

In [86]:
df

,plan_idx,title,question_idx,steps,question_type,step_pair_idx_asked_about,binary_question,why_question,model_name,prompt_key,model_input,model_answer,binary_answer,why_answer
0,710,spiced-guava-cake,710,"[Preheat oven to 180 C / Gas 4., Grease and fl...",nondependent_real_before,"[3, 4]",Must Step 4 happen before Step 5?,Why must Step 4 happen before Step 5?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","{\n ""binary_answer"": ""no"",\n ""why_answer"": ""...",no,Step 4 and Step 5 involve separate ingredients...
1,711,chocolate-chocolate-cake,711,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_before,"[1, 5]",Must Step 2 happen before Step 6?,Why must Step 2 happen before Step 6?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_...",yes,Step 2 involves melting the chocolate which ne...
2,712,chicken-prawn-and-rice-soup,712,"[Mix together the prawns, chicken, egg and cor...",nondependent_real_before,"[3, 10]",Must Step 4 happen before Step 11?,Why must Step 4 happen before Step 11?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","{\n ""binary_answer"": ""no"",\n ""why_answer"": ""...",no,Step 4 and Step 11 are part of different sub-p...
3,713,green-tofu-curry,713,"[Place water in a medium saucepan, and stir in...",nondependent_real_before,"[0, 4]",Must Step 1 happen before Step 5?,Why must Step 1 happen before Step 5?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""no"",\n ""why_a...",no,Step 1 and Step 5 involve different components...
4,714,very-vanilla-ry-cheesecake,714,[Preheat oven to 150 C / Gas 2 and lightly but...,nondependent_real_before,"[4, 7]",Must Step 5 happen before Step 8?,Why must Step 5 happen before Step 8?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""yes"",\n ""why_...",yes,Step 5 involves preparing the crust which must...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,1415,chocolate-chocolate-cake,1415,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_after,"[2, 18]",Must Step 19 happen after Step 3?,Why must Step 19 happen after Step 3?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""no"",\n ""why_a...",no,Step 19 refers to a potential outcome during t...
1416,1416,chocolate-chocolate-cake,1416,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_after,"[5, 18]",Must Step 19 happen after Step 6?,Why must Step 19 happen after Step 6?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""no"",\n ""why_a...",no,Step 19 refers to a characteristic of the cake...
1417,1417,chicken and vegetable pie,1417,"[Preheat oven to 220 C / Gas mark 7., In a sau...",nondependent_real_after,"[3, 5]",Must Step 6 happen after Step 4?,Why must Step 6 happen after Step 4?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","{\n ""binary_answer"": ""yes"",\n ""why_answer"": ...",yes,Step 6 involves cooking additional ingredients...
1418,1418,chocolate-chocolate-cake,1418,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_after,"[10, 14]",Must Step 15 happen after Step 11?,Why must Step 15 happen after Step 11?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...","```json\n{\n ""binary_answer"": ""no"",\n ""why_a...",no,"Step 15, which involves preparing the cake tin..."


In [87]:
df_copy = df.copy()

In [88]:
df_copy = df_copy.drop('model_answer', axis=1)

In [89]:
df_copy

,plan_idx,title,question_idx,steps,question_type,step_pair_idx_asked_about,binary_question,why_question,model_name,prompt_key,model_input,binary_answer,why_answer
0,710,spiced-guava-cake,710,"[Preheat oven to 180 C / Gas 4., Grease and fl...",nondependent_real_before,"[3, 4]",Must Step 4 happen before Step 5?,Why must Step 4 happen before Step 5?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 4 and Step 5 involve separate ingredients...
1,711,chocolate-chocolate-cake,711,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_before,"[1, 5]",Must Step 2 happen before Step 6?,Why must Step 2 happen before Step 6?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,Step 2 involves melting the chocolate which ne...
2,712,chicken-prawn-and-rice-soup,712,"[Mix together the prawns, chicken, egg and cor...",nondependent_real_before,"[3, 10]",Must Step 4 happen before Step 11?,Why must Step 4 happen before Step 11?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 4 and Step 11 are part of different sub-p...
3,713,green-tofu-curry,713,"[Place water in a medium saucepan, and stir in...",nondependent_real_before,"[0, 4]",Must Step 1 happen before Step 5?,Why must Step 1 happen before Step 5?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 1 and Step 5 involve different components...
4,714,very-vanilla-ry-cheesecake,714,[Preheat oven to 150 C / Gas 2 and lightly but...,nondependent_real_before,"[4, 7]",Must Step 5 happen before Step 8?,Why must Step 5 happen before Step 8?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,Step 5 involves preparing the crust which must...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,1415,chocolate-chocolate-cake,1415,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_after,"[2, 18]",Must Step 19 happen after Step 3?,Why must Step 19 happen after Step 3?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 19 refers to a potential outcome during t...
1416,1416,chocolate-chocolate-cake,1416,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_after,"[5, 18]",Must Step 19 happen after Step 6?,Why must Step 19 happen after Step 6?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 19 refers to a characteristic of the cake...
1417,1417,chicken and vegetable pie,1417,"[Preheat oven to 220 C / Gas mark 7., In a sau...",nondependent_real_after,"[3, 5]",Must Step 6 happen after Step 4?,Why must Step 6 happen after Step 4?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,Step 6 involves cooking additional ingredients...
1418,1418,chocolate-chocolate-cake,1418,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_after,"[10, 14]",Must Step 15 happen after Step 11?,Why must Step 15 happen after Step 11?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,"Step 15, which involves preparing the cake tin..."


In [90]:
counts = df_copy['binary_answer'].value_counts()
num_yes = counts['yes']
num_no = counts['no']

In [91]:
num_yes

581

In [92]:
num_no

839

In [93]:
df_yes = df_copy[df_copy['binary_answer'] == 'yes']

In [94]:
df_yes

,plan_idx,title,question_idx,steps,question_type,step_pair_idx_asked_about,binary_question,why_question,model_name,prompt_key,model_input,binary_answer,why_answer
1,711,chocolate-chocolate-cake,711,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_before,"[1, 5]",Must Step 2 happen before Step 6?,Why must Step 2 happen before Step 6?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,Step 2 involves melting the chocolate which ne...
4,714,very-vanilla-ry-cheesecake,714,[Preheat oven to 150 C / Gas 2 and lightly but...,nondependent_real_before,"[4, 7]",Must Step 5 happen before Step 8?,Why must Step 5 happen before Step 8?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,Step 5 involves preparing the crust which must...
5,715,smoked-salmon-and-grapefruit-salad,715,"[Top and tail the grapefruits., Slice the skin...",nondependent_real_before,"[1, 4]",Must Step 2 happen before Step 5?,Why must Step 2 happen before Step 5?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,"Step 2 involves preparing the grapefruit, whic..."
6,716,fusilli-and-meatballs,716,[Coat a large saucepan with cooking spray or a...,nondependent_real_before,"[5, 15]",Must Step 6 happen before Step 16?,Why must Step 6 happen before Step 16?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,Step 6 involves adding the processed tomatoes ...
9,719,tangy-gammon-in-cider,719,"[Preheat the oven to 200 degrees /gas mark 6.,...",nondependent_real_before,"[0, 6]",Must Step 1 happen before Step 7?,Why must Step 1 happen before Step 7?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,Preheating the oven in Step 1 is necessary bef...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,1406,tortellini-fritto-with-almond-butter,1406,"[Bring a saucepan of water to the boil., Add t...",nondependent_real_after,"[5, 7]",Must Step 8 happen after Step 6?,Why must Step 8 happen after Step 6?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,"Step 8 involves enjoying the prepared dish, wh..."
1407,1407,fusilli-and-meatballs,1407,[Coat a large saucepan with cooking spray or a...,nondependent_real_after,"[11, 13]",Must Step 14 happen after Step 12?,Why must Step 14 happen after Step 12?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,Step 14 must happen after Step 12 because the ...
1411,1411,chicken and vegetable pie,1411,"[Preheat oven to 220 C / Gas mark 7., In a sau...",nondependent_real_after,"[3, 6]",Must Step 7 happen after Step 4?,Why must Step 7 happen after Step 4?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,Step 7 involves adding ingredients to the cook...
1413,1413,spiced butternut squash soup,1413,"[Preheat the oven to 190 C / Gas mark 6., Pour...",nondependent_real_after,"[3, 10]",Must Step 11 happen after Step 4?,Why must Step 11 happen after Step 4?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,"Step 11 involves cooking potatoes in the soup,..."


In [95]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

In [96]:
why_answer = df_yes['why_answer'].tolist()

In [97]:
# Create a bag of words representation
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(why_answer)

In [98]:
#Perform K-means clustering
kmeans = KMeans(n_clusters=4)  # Adjust the number of clusters as needed
kmeans.fit(X)

KMeans(n_clusters=4)

In [99]:
# Add cluster labels
df_yes['cluster'] = kmeans.labels_

/var/folders/j9/5zmzdnn93jj4z_0j5tw59c540000gn/T/ipykernel_39006/3903029983.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yes['cluster'] = kmeans.labels_


In [100]:
df_yes

,plan_idx,title,question_idx,steps,question_type,step_pair_idx_asked_about,binary_question,why_question,model_name,prompt_key,model_input,binary_answer,why_answer,cluster
1,711,chocolate-chocolate-cake,711,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_before,"[1, 5]",Must Step 2 happen before Step 6?,Why must Step 2 happen before Step 6?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,Step 2 involves melting the chocolate which ne...,1
4,714,very-vanilla-ry-cheesecake,714,[Preheat oven to 150 C / Gas 2 and lightly but...,nondependent_real_before,"[4, 7]",Must Step 5 happen before Step 8?,Why must Step 5 happen before Step 8?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,Step 5 involves preparing the crust which must...,0
5,715,smoked-salmon-and-grapefruit-salad,715,"[Top and tail the grapefruits., Slice the skin...",nondependent_real_before,"[1, 4]",Must Step 2 happen before Step 5?,Why must Step 2 happen before Step 5?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,"Step 2 involves preparing the grapefruit, whic...",1
6,716,fusilli-and-meatballs,716,[Coat a large saucepan with cooking spray or a...,nondependent_real_before,"[5, 15]",Must Step 6 happen before Step 16?,Why must Step 6 happen before Step 16?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,Step 6 involves adding the processed tomatoes ...,1
9,719,tangy-gammon-in-cider,719,"[Preheat the oven to 200 degrees /gas mark 6.,...",nondependent_real_before,"[0, 6]",Must Step 1 happen before Step 7?,Why must Step 1 happen before Step 7?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,Preheating the oven in Step 1 is necessary bef...,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,1406,tortellini-fritto-with-almond-butter,1406,"[Bring a saucepan of water to the boil., Add t...",nondependent_real_after,"[5, 7]",Must Step 8 happen after Step 6?,Why must Step 8 happen after Step 6?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,"Step 8 involves enjoying the prepared dish, wh...",0
1407,1407,fusilli-and-meatballs,1407,[Coat a large saucepan with cooking spray or a...,nondependent_real_after,"[11, 13]",Must Step 14 happen after Step 12?,Why must Step 14 happen after Step 12?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,Step 14 must happen after Step 12 because the ...,0
1411,1411,chicken and vegetable pie,1411,"[Preheat oven to 220 C / Gas mark 7., In a sau...",nondependent_real_after,"[3, 6]",Must Step 7 happen after Step 4?,Why must Step 7 happen after Step 4?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,Step 7 involves adding ingredients to the cook...,0
1413,1413,spiced butternut squash soup,1413,"[Preheat the oven to 190 C / Gas mark 6., Pour...",nondependent_real_after,"[3, 10]",Must Step 11 happen after Step 4?,Why must Step 11 happen after Step 4?,gpt-4-turbo-2024-04-09,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",yes,"Step 11 involves cooking potatoes in the soup,...",0


In [101]:
df_yes.to_csv('nondependent_real_basic_binary_explain_yes.csv', index=False)

In [57]:
#rows_with_label = df_yes[df_yes['cluster'] == 0]